## Important Dependencies with their required versions


In [ ]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb


## Imports and Setup

In [5]:
# from langchain_community.llms import HuggingFaceHub
# from langchain_community.chat_models.huggingface import ChatHuggingFace
import re
import os
from Model import InitializeModel,InitializeGptModel
from GenerateUnitTest import InitializeTestChain,queryGpt
from utils.PreprocessUtils import *
from utils.FeedbackUtils import *
from utils.LLMUtilis import *
from utils.FuncUtils import *
from dotenv import load_dotenv
load_dotenv()
# from huggingface_hub import notebook_login
# notebook_login()
import pandas as pd    
from DB import *

HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
import sys
from io import StringIO
import sys
from io import StringIO
from CustomThread import CustomThread

def runCode(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    oldstderr = sys.stderr
    redirected_output2 = sys.stderr = StringIO()
    result = ""
    thread = CustomThread()
    try:
        thread.start()
        exec(code, globals())
        result = redirected_output.getvalue()
    except Exception as e:
        # print(repr(e))
        result = repr(e)
    except SystemExit as s:
        # print(repr(s))
        result = redirected_output2.getvalue()
    except KeyboardInterrupt as k:
        result = "timed out"
    thread.stop()
    globals().popitem()
    sys.stdout = old_stdout
    sys.stderr = oldstderr
    # print(result)
    return result

## Initialize Agent

In [6]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     GPTNeoForCausalLM,
# )
# from tokenizers.implementations import ByteLevelBPETokenizer
# tokenizer = ByteLevelBPETokenizer(
#     "vocab.json",
#     "merges.txt",
# )
load_dotenv()
model_id="google/gemma-7b-it"
max_new_tokens= 6000
if 'Mixtral' in model_id:
    max_new_tokens= 20000

llm, chat_model = InitializeModel(os.environ['HF_TOKEN'], model_id,
                                  max_new_tokens= max_new_tokens)
if 'Mixtral' in model_id:
    GenUnitTestChain= InitializeTestChain(llm)
else:
    GenUnitTestChain= InitializeTestChain(chat_model)

HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
db= connect_db()
# llm2,llm3=InitializeGptModel(os.environ['OPENAI_API_KEY'])


WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


## Generate Unit Tests from Code

In [7]:

total_test_cases = 0
total_success_test_cases = 0
failed_test_cases = 0
num_failed_examples = 0
num_successful_examples = 0
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]
TotalCases=0
TotalFailedCases=0
incompleteResponses=0
try:
  with open("RunningLogs.xlsx"):
    df = pd.read_excel("RunningLogs.xlsx",engine='openpyxl')
except FileNotFoundError:
  # Create a file with the name RunningLogs.xlsx
    dummy = pd.DataFrame({'Description':[],'Code':[],'GeneratedCode':[],'Feedback':[]})
    # with open("RunningLogs.xlsx", 'w') as fp:
    #     pass
    dummy.to_excel("RunningLogs.xlsx",index=False)
    df = pd.read_excel("RunningLogs.xlsx",engine='openpyxl')


for i in range(0,2):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    codes_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessStringFewShot(codes_of_few_shot, test_cases_few_shot)
    # print("fewShotStr is :",fewShotStr)
    unittest=GenUnitTestChain.invoke({"description":description,"code":code,"test_cases_of_few_shot":fewShotStr})#,"test_cases_of_few_shot":fewShotStr
    ############################################

    ### get the response (unit test) code only without natural language
    unittest_code,isIncompleteResponse = get_code_from_response(unittest['text'],funcDefiniton)
    if(isIncompleteResponse):
        incompleteResponses+=1
    #print("unittest_code is ",unittest_code)
    unittest_code = replaceUnitTestCall(unittest_code)
    # codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    codeTobeRun = get_running_code(code,unittest_code)
    # print("unittest_code is: \n ",unittest_code)
    # print("run_code is \n",codeTobeRun)
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)

    
    if(feedbackparsed==''):
        #in case of OK
        num_successful_examples +=1
        OKCases+=1
        print(f"Test example {i+1} succeeded" )
        num_of_assertions = get_num_assertions(unittest_code)
        print("Number of Ran Tests : " , num_of_assertions) 
        print("Number of Succeeded Test : ", num_of_assertions)
        print("Number of Succeeded Test : ", 0)
        total_test_cases += num_of_assertions

    else:
         num_failed_examples +=1
         total_cases, failed_cases = get_failed_testcases(feedback)
         print(f"Test example {i+1} failed" )
         print("Number of Ran Tests : " , total_cases) 
         print("Number of failed Tests : ", failed_cases)
         print("Number of Succeeded Test : ", total_cases - failed_cases)
         total_test_cases += total_cases
         failed_test_cases+= failed_cases
        

    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    new_row=pd.DataFrame({'Description':description,'Code':code,'GeneratedCode':unittest_code,'Feedback':feedbackparsed},index=[0])
    df = df.append(new_row, ignore_index=True)
    # Save the updated DataFrame back to the excel file using 'openpyxl' engine for writing
    with pd.ExcelWriter("RunningLogs.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:
        # df.to_excel(writer, sheet_name=df.sheet_name.tolist()[0], index=False)  # Assuming only one sheet
        df.to_excel(writer, index=False)  # Assuming only one sheet





#dump data into xlsx file
resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
#resDF.to_excel("GeneratedFeedback.xlsx",index=False)
resDF.to_json("GeneratedFeedback.json",orient='records')
# print("OKCases are ",OKCases)
print('\n End of Test Cases \n')

print("Total secceeded examples : " ,num_successful_examples)
print("Total failed examples : ", num_failed_examples)


print("Total  testcases : " ,total_test_cases)
print("Total succeeded testcases : ",total_test_cases- failed_test_cases)
print("Total failed testcases : ",failed_test_cases)
with open('Res.txt', 'w') as f:
    f.write("OKCases are "+str(OKCases)+"\n")
    f.write("Total Cases are "+str(len(descriptions))+"\n")

print("Incomplete Responses are ",incompleteResponses)

Running Test Case  1


NameError: name 'get_running_code' is not defined

## GPT MODEL

In [ ]:
introCode = """
import sys
sys.setrecursionlimit(100)
"""
import time
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]

for i in range(0,10):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    code_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessStringFewShot(code_of_few_shot, test_cases_few_shot)

    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model
    #try gpt model
    unittest=queryGpt(llm2,description,code,fewshots=False,test_cases_of_few_shot=fewShotStr)['choices'][0]['message']['content']
    print("gpt_response is ",unittest)

    
    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model

    ### get the response (unit test) code only without natural language
    unittest_code = get_code_from_response(unittest,funcDefiniton)
    unittest_code = replaceUnitTestCall(unittest_code)
    ##############run code with the original and unit test concatenated #######################
    codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    print("run_code is")
    print(codeTobeRun)
    # print("------------------------------lolooloolololo ------------------------------------")
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    if(feedbackparsed==''):
        #in case of OK
        OKCases+=1
        # feedbackparsed=""
        # for i,line in enumerate(feedback.split('\n')):
        #     if i>1:
        #         feedbackparsed+=line+'\n'
    #print(feedbackparsed)
    print(feedback)
    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    time.sleep(20)

#dump data into xlsx file
resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
#resDF.to_excel("GeneratedFeedbackGPT.xlsx",index=False)
resDF.to_json("GeneratedFeedbackGPT.json",orient='records')
print("OKCases are ",OKCases)
print("Total Cases are ",len(descriptions))
with open('ResGPT.txt', 'w') as f:
    f.write("OKCases are "+str(OKCases)+"\n")
    f.write("Total Cases are "+str(len(descriptions))+"\n")


## Generate Unit Tests using Few Shots examples

In [ ]:
description="""def is_nested(string): ''' Create a function that takes a string as input which contains only square brackets. The function should return True if and only if there is a valid subsequence of brackets where at least one bracket in the subsequence is nested. is_nested('[[]]') ➞ True is_nested('[]]]]]]][[[[[]') ➞ False is_nested('[][]') ➞ False is_nested('[]') ➞ False is_nested('[[][]]') ➞ True is_nested('[[]][[') ➞ True '''"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""

Gen_UnitTest_with_FewShots_template="""Task: Given the following description and python code:

Description:
{description}
Code:
'''python
{code}'''

Generate a class that contains multiple unit tests written in python to find bugs, syntax errors or logical errors in the code to conform with the description and include any required imports.
You are given example unit tests of a similar code from a database:
'''python
{test_cases_of_few_shot}'''


Generate a class that contains multiple unit tests written in python to find bugs, syntax errors or logical errors in the code to conform with the description and include any required imports.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":"""
# Generate multiple unit tests in python code in form of functions to find bugs, syntax errors or logical errors and if the code functionality does not comply with the description.

Generate_Unit_Tests_FewShots__Template = PromptTemplate(
    template=Gen_UnitTest_with_FewShots_template,
    input_variables=["description","code","test_cases_of_few_shot"],
    verbose=True
    )

GenUnitTestChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_Unit_Tests_FewShots__Template
)
res=GenUnitTestChain.invoke({
                             "description":description,
                             "code":code,
                             "test_cases_of_few_shot":test_cases_of_few_shot
                             })
print(res['text'])

In [ ]:
#made according to mixtral response
def get_code_from_response(response):
    lines=response.split("\n")
    code=""
    in_code=False
    in_new_response=False
    for i,line in enumerate(lines):
        if in_new_response:
          if line.startswith("```python"):
            in_code=True
          elif line.startswith("```"):
            in_code=False
            return code
          elif in_code==True:
            if(i==len(lines)-1):
              return code
            code+=line+"\n"
        else:
            if "[/INST]" in line:
                in_new_response=True
                # case where the llm only wrote a code instead of a full response with other text
                if "```python" in line :
                    in_code=True
    #incomplete output, most likely incomplete assertion.
    # ignore this last assertion and move with the rest

    return code
# replaces the unittest call with another one to fix exec problem with running on colab
def replaceUnitTestCall(code):
    pattern = r"unittest.main()"

    # Use re.sub() to replace the pattern with the replacement string
    modified_code = re.sub(pattern, "unittest.main(argv=['first-arg-is-ignored'])", code)

    return modified_code



res_code= get_code_from_response(res['text'])
res_code= replaceUnitTestCall(res_code)
print("res_code is")
print(res_code)

In [ ]:
print(runCode(code+"\n"+res_code))

In [ ]:
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
falsecode="""import unittest

class TestIsNested(unittest.TestCase):

    def test_simple_cases(self):
        self.assertTalse(is_nested('[[]]'))
        self.assertFalse(is_nested('[]]]]]]][[[[[]'))
        self.assertFalse(is_nested('[][]'))
        self.assertFalse(is_nested('[]'))
        self.assertTrue(is_nested('[[[[]]]]'))
        self.assertFalse(is_nested('[]]]]]]]]]]'))
        self.assertTrue(is_nested('[][][[]]'))
        self.assertFalse(is_nested('[[]'))
        self.assertFalse(is_nested('[]]'))
        self.assertTrue(is_nested('[[]][['))
        self.assertTrue(is_nested('[[][]]'))

    def test_edge_cases(self):
        self.assertTrue(is_nested(''))
        self.assertFalse(is_nested('[[[[[[[['))
        self.assertFalse(is_nested(']]]]]]]]'))

if __name__ == '__main__':
    # unittest.main(argv=['first-arg-is-ignored' ])()
    # unittest.main()
"""
from utils.LLMUtilis import *
feedback=runCode(code+"\n"+falsecode)
print(feedback)

## Bug Fixing

In [ ]:
BugRepair_template_str = """Act as a python expert in bug fixing and assume that I need to fix all errors perfectly.
I am going to give you the bugs encountered due to running test cases then after
that I will give you the code under test with bugs so that you fix it, the line numbers of
the error correspond to the line that the error occured in the code provided.
The following is the code under test:
```python
{code}```

The following are the bugs and test cases corresponding to the bugs:
```python
{TestcaseCode}```

The result after running these test cases on an interpreter:
{InterpreterFeedback}

Change either the code or the test cases according to the feedback of the interpreter.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
"""
InterpreterFeedback=feedbackparsed

BugRepair_Template = PromptTemplate(
    template=BugRepair_template_str,
    input_variables=["code","TestcaseCode","InterpreterFeedback"],
    verbose=True
    )

BugFixChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=BugRepair_Template
)

res=BugFixChain.invoke({
                             "code":code,
                             "TestcaseCode":res_code,
                             "InterpreterFeedback":InterpreterFeedback
                             })
print(res['text'])

## input code preprocessor

In [ ]:
#if the input code is without description, we want to generate description
Generate_description_Template = """Act as a python expert, Describe the purpose and functionality of the following Python code.

```python
{code}```
The output should be like that:
The provided Python code aims to [explain purpose].
In summary, [concise summary of functionality ].
Examples: [3 or 4 input output examples]
"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
Generate_description_PromptTemplate = PromptTemplate(
    template=Generate_description_Template,
    input_variables=["code"],
    verbose=True
    )

GenDescriptionChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_description_PromptTemplate
)
res=GenDescriptionChain.invoke({
                             "code":code
                             })
print(res['text'])

In [ ]:
def get_num_failed_testcases(feedback):
    # Get the total number of test cases
    total_cases = re.search(r'Ran (\d+) tests', feedback).group(1)

    # Get the number of failed test cases
    failed_cases = re.search(r'failures=(\d+)', feedback).group(1)
    return total_cases, failed_cases